## Curso de Investigación para Radiología

* Héctor Henríquez Leighton MD, MS
* hhenriquez@miuandes.cl

In [ ]:
! pip install SimpleITK
! pip install pydicom

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pydicom
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import requests
import json

## Suprimir advertencias
import warnings
warnings.filterwarnings('ignore')

## Descarga de token de kaggle que permite acceder al set de datos
json_response= requests.get("https://raw.github.com/HectorHenriquez/Airway_segmentation/main/kaggle.json")
token = json.loads(json_response.text)
with open("kaggle.json", "w") as outfile:
    json.dump(token, outfile)

## Carga de datos desde Kaggle
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## Dataset
! kaggle datasets download hshenriquez/curso-investigacion-imagenes
!unzip /content/curso-investigacion-imagenes

In [ ]:
def show_slice_window(slice, level, window):
    """
    input: imagen array 2D,
    permite ajustar ventana y nivel para mejorar contraste de la imagen.
    output: imagen array 2D ventaneada.
   """
    max = level + window/2
    min = level - window/2
    slice = slice.clip(min,max)
    return(slice)

### Leer archivo DICOM

In [ ]:
# Cargar el archivo DICOM
ds = pydicom.dcmread("/content/rx_anon.dcm")
print(ds)

In [ ]:
### Acceder a campos específicos:
print("Nombre:", ds.PatientName)
print("KVP:", ds.KVP)
print("Exposure Time:", ds.ExposureTime)
print("Tube Current:", ds.XRayTubeCurrent)

In [ ]:
## Acceder a la imagen:
pixel_array = ds.pixel_array
print(pixel_array.shape)

In [ ]:
## Valores máximos, mínimos y promedio
print(pixel_array.max())
print(pixel_array.min())
print(pixel_array.mean())

In [ ]:
plt.hist(pixel_array.ravel(), bins=256, color='dodgerblue', alpha=0.7)
plt.title("Histograma de intensidades")
plt.xlabel("Valor de píxel")
plt.ylabel("Frecuencia")
plt.grid(True)

In [ ]:
level = 1800
width = 1000

plt.figure(figsize=(10,10))
plt.imshow(pixel_array, cmap='gray')
#plt.imshow(show_slice_window(pixel_array,level,width), cmap='gray')
plt.title("Rx array")
plt.show()

In [ ]:

plt.hist(show_slice_window(pixel_array,level,width).ravel(), bins=256, color='dodgerblue', alpha=0.7)
plt.title("Histograma de intensidades")
plt.xlabel("Valor de píxel")
plt.ylabel("Frecuencia")
plt.grid(True)

In [ ]:
plt.figure()
plt.imshow(pixel_array[0:500,0:500], cmap='gray')
plt.title("Rx array")
plt.show()

### Leer archivos volumétricos

In [ ]:
volume = sitk.ReadImage("/content/imaging.nii")
mask =  sitk.ReadImage("/content/segmentation.nii")

In [ ]:
### Propiedades de la imagen

print("Dimensiones:", volume.GetSize())
print("Spacing:", volume.GetSpacing())
print("Dirección:", volume.GetDirection())

In [ ]:

print("Dimensiones:", mask.GetSize())
print("Spacing:", mask.GetSpacing())
print("Dirección:", mask.GetDirection())

In [ ]:
### Acceder a los pixeles:

volume_array = sitk.GetArrayFromImage(volume)
mask_array = sitk.GetArrayFromImage(mask)
print(volume_array.shape)
print(mask_array.shape)

In [ ]:
## Valores imagen:

print(volume_array.max())
print(volume_array.min())

## valores máscara:
print(mask_array.max())
print(mask_array.min())

In [ ]:
## histograma
plt.hist(volume_array.ravel(), bins=256, color='salmon', alpha=0.7)
plt.title("Histograma de intensidades")
plt.xlabel("Valor de píxel")
plt.ylabel("Frecuencia")
plt.grid(True)

In [ ]:
## Visualización

corte_axial=180

plt.figure()
plt.imshow(np.rot90(volume_array[:,:,corte_axial], k = -1), cmap='gray')
plt.imshow(np.rot90(mask_array[:,:,corte_axial], k = -1), cmap='jet', alpha=0.3)
plt.show()

In [ ]:

def visualize_3d_mask_labels_sampled(mask, sample_frac=0.1, alpha=0.5, title=""):
    zs, ys, xs = np.where(mask > 0)
    vals = mask[zs, ys, xs]
    # muestreo
    n = len(zs)
    idx = np.random.choice(n, size=int(n*sample_frac), replace=False)
    zs, ys, xs, vals = zs[idx], ys[idx], xs[idx], vals[idx]

    fig = go.Figure()
    for lbl in np.unique(vals):
        sel = vals == lbl
        fig.add_trace(go.Scatter3d(
            x=xs[sel], y=ys[sel], z=zs[sel],
            mode='markers',
            name=f"Label {lbl}",
            marker=dict(size=2, opacity=alpha)
        ))
    fig.update_layout(title=title,
                      scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
    fig.show()

In [ ]:
visualize_3d_mask_labels_sampled(mask_array, sample_frac=0.1, alpha=0.5, title="Segmentación 3D")